In [2]:
import torch
import ssl
import matplotlib.pyplot as plt
from src.logger.logger_initializer import LoggerInitializer

ssl._create_default_https_context = ssl._create_unverified_context
device = 'cuda'
torch.__version__

LoggerInitializer().init()

[2023-04-17 01:30:40] 5912 root {logger_initializer-47} INFO - Current process ID: 5912


True

In [3]:
from src.data_loader.dataset_getter import DatasetGetter
dataset = DatasetGetter('../../bayesian_test/data', batch_size=12).get()

In [4]:
from src.models.cnn.resnet50_cnn_classifier import Resnet50CnnClassifier
classifier = Resnet50CnnClassifier(dataset=dataset, device=device).init()

In [5]:
result_df, result_dict = classifier.fit(num_epoch=2)
display(result_df)
print(result_dict)

[2023-04-17 01:30:53] 5912 src.scoring.epoch_score_printer {epoch_score_printer-21} INFO - [Epoch 0/2]: f1_score: 0.701 accuracy_score: 0.727 loss: 186.333 
[2023-04-17 01:30:59] 5912 src.scoring.epoch_score_printer {epoch_score_printer-21} INFO - [Epoch 1/2]: f1_score: 0.720 accuracy_score: 0.742 loss: 118.075 


,f1_score,accuracy_score,loss
0,0.701435,0.726562,186.332771
1,0.720311,0.742188,118.074509


{'best_loss': 118.07450944185257, 'best_f1_score': 0.7203111119224076, 'best_accuracy_score': 0.7421875, 'best_loss_model': OrderedDict([('conv1.weight', tensor([[[[-7.4457e-03, -3.1783e-03,  3.7353e-02,  ...,  4.7936e-02,
           -2.0325e-02,  8.8140e-03],
          [-5.7435e-02,  4.4709e-02,  7.7509e-02,  ...,  8.8442e-02,
            2.9346e-02, -5.8331e-02],
          [ 6.8356e-02, -2.7044e-01,  4.0348e-01,  ..., -1.6491e-01,
            2.1868e-01, -7.2909e-02],
          ...,
          [-1.0874e-01,  3.8148e-01, -4.5487e-01,  ...,  6.8366e-01,
           -5.7855e-01,  2.2461e-01],
          [ 2.5698e-02, -1.7703e-01,  6.4375e-01,  ...,  5.2644e-01,
           -4.9317e-02, -6.8082e-02],
          [ 4.5281e-02, -1.3072e-01,  1.7864e-02,  ..., -3.5753e-01,
            1.8976e-01, -2.2302e-02]],

         [[ 8.9197e-03,  4.8768e-03, -1.5356e-02,  ...,  8.6949e-02,
           -6.5541e-02,  1.6895e-02],
          [-3.3543e-02,  3.9067e-02,  8.6565e-02,  ...,  1.1713e-01,
           

In [6]:
classifier.save_model(result_dict)

In [7]:
test_model_weights_path = '../../bayesian_refactoring/assets/model_weights/resnet50cnn_5912_813dec81-d9db-43f9-8e68-a4792822a394.pt'

In [8]:
test_results = torch.load(test_model_weights_path)

In [10]:
list(test_results.keys())

['best_loss',
 'best_f1_score',
 'best_accuracy_score',
 'best_loss_model',
 'best_f1_score_model',
 'best_accuracy_score_model']

In [9]:
new_classifier = Resnet50CnnClassifier(dataset=dataset, device=device).load_model(model_state_dict=test_results['best_f1_score_model'])

In [10]:
y_true, y_pred = new_classifier.predict(dataset.validate_dataloader)

In [11]:
new_classifier.score(dataset.validate_dataloader)

{'f1_score': 0.7022692266004554, 'accuracy_score': 0.7351778656126482}